# Create Gates Foundation Awards

Creates awards from the Bill & Melinda Gates Foundation committed grants database. ~40K grants.

**Prerequisites:**
- Run `scripts/local/gates_to_s3.py` to download and upload the data first.

**Data source:** https://www.gatesfoundation.org/about/committed-grants  
**S3 location:** `s3a://openalex-ingest/awards/gates/gates_projects.parquet`

**Gates Foundation funder:**
- funder_id: 4320306137
- display_name: "Bill and Melinda Gates Foundation"
- ROR: https://ror.org/0456r8d26
- DOI: 10.13039/100000865

**Notes:**
- Gates Foundation grants are to **organizations**, not individual researchers
- The grantee is typically an institution (university, NGO, etc.)
- No PI information is available in the public grants database
- Division field maps to funder_scheme (e.g., "Global Health", "Global Development")

## Step 1: Create Staging Table from S3

In [ ]:
%sql
-- Create the staging table from S3 parquet
CREATE OR REPLACE TABLE openalex.awards.gates_raw
USING delta
AS
SELECT
    *,
    current_timestamp() as databricks_ingested_at
FROM parquet.`s3a://openalex-ingest/awards/gates/gates_projects.parquet`;

In [ ]:
%sql
-- Check row count (should be ~40K)
SELECT COUNT(*) as total_grants FROM openalex.awards.gates_raw;

In [ ]:
%sql
-- Sample the raw data
SELECT * FROM openalex.awards.gates_raw LIMIT 5;

## Step 2: Create Gates Awards Table

In [ ]:
%sql
CREATE OR REPLACE TABLE openalex.awards.gates_awards
USING delta
AS
WITH
-- Get Gates Foundation funder from OpenAlex by explicit funder_id
gates_funder AS (
    SELECT
        funder_id,
        display_name,
        ror_id,
        doi
    FROM openalex.common.funder
    WHERE funder_id = 4320306137  -- Bill and Melinda Gates Foundation
),

awards_transformed AS (
    SELECT
        -- Generate unique ID: xxhash64 of funder_id:grant_id
        abs(xxhash64(CONCAT(f.funder_id, ':', LOWER(g.grant_id)))) % 9000000000 as id,

        -- Display name = grantee_name + purpose (truncated)
        -- Gates grants don't have titles, so we use grantee + purpose summary
        CONCAT(
            g.grantee_name,
            ' - ',
            CASE
                WHEN LENGTH(g.purpose) > 150 THEN CONCAT(SUBSTRING(g.purpose, 1, 147), '...')
                ELSE g.purpose
            END
        ) as display_name,

        -- Description = full purpose statement
        g.purpose as description,

        -- Funder info
        f.funder_id,
        g.grant_id as funder_award_id,

        -- Amount in USD
        CAST(g.amount AS DOUBLE) as amount,
        'USD' as currency,

        -- Funder struct
        struct(
            CONCAT('https://openalex.org/F', f.funder_id) as id,
            f.display_name,
            f.ror_id,
            f.doi
        ) as funder,

        -- Funding type - Gates Foundation primarily makes grants
        -- Map divisions/topics to funding types where possible
        CASE
            WHEN LOWER(g.topic) LIKE '%fellowship%' THEN 'fellowship'
            WHEN LOWER(g.topic) LIKE '%scholarship%' THEN 'fellowship'
            WHEN LOWER(g.topic) LIKE '%training%' THEN 'training'
            WHEN LOWER(g.topic) LIKE '%research%' THEN 'research'
            WHEN LOWER(g.topic) LIKE '%discovery%' THEN 'research'
            WHEN LOWER(g.topic) LIKE '%translational%' THEN 'research'
            WHEN LOWER(g.division) LIKE '%global health%' THEN 'research'
            WHEN LOWER(g.topic) LIKE '%education%' THEN 'training'
            WHEN LOWER(g.topic) LIKE '%infrastructure%' THEN 'infrastructure'
            ELSE 'grant'
        END as funding_type,

        -- Funder scheme = division (e.g., "Global Health", "Global Development")
        g.division as funder_scheme,

        -- Provenance
        'gates_foundation' as provenance,

        -- Dates (stored as strings in YYYY-MM-DD format)
        TRY_TO_DATE(g.start_date, 'yyyy-MM-dd') as start_date,
        TRY_TO_DATE(g.end_date, 'yyyy-MM-dd') as end_date,
        YEAR(TRY_TO_DATE(g.start_date, 'yyyy-MM-dd')) as start_year,
        YEAR(TRY_TO_DATE(g.end_date, 'yyyy-MM-dd')) as end_year,

        -- Lead investigator - Gates grants go to organizations, not individuals
        -- We store the grantee organization in the affiliation field
        CASE
            WHEN g.grantee_name IS NOT NULL THEN
                struct(
                    CAST(NULL AS STRING) as given_name,
                    CAST(NULL AS STRING) as family_name,
                    CAST(NULL AS STRING) as orcid,
                    CAST(NULL AS DATE) as role_start,
                    struct(
                        g.grantee_name as name,
                        g.grantee_country as country,
                        CAST(NULL AS ARRAY<STRUCT<id:STRING, type:STRING, asserted_by:STRING>>) as ids
                    ) as affiliation
                )
            ELSE NULL
        END as lead_investigator,

        -- Co-lead and other investigators (not available)
        CAST(NULL AS STRUCT<
            given_name:STRING,
            family_name:STRING,
            orcid:STRING,
            role_start:DATE,
            affiliation:STRUCT<name:STRING, country:STRING, ids:ARRAY<STRUCT<id:STRING, type:STRING, asserted_by:STRING>>>
        >) as co_lead_investigator,

        CAST(NULL AS ARRAY<STRUCT<
            given_name:STRING,
            family_name:STRING,
            orcid:STRING,
            role_start:DATE,
            affiliation:STRUCT<name:STRING, country:STRING, ids:ARRAY<STRUCT<id:STRING, type:STRING, asserted_by:STRING>>>
        >>) as investigators,

        -- Landing page URL - Gates committed grants page
        CONCAT('https://www.gatesfoundation.org/about/committed-grants?q=', g.grant_id) as landing_page_url,

        -- No DOI for Gates grants
        CAST(NULL AS STRING) as doi,

        -- Works API URL
        concat('https://api.openalex.org/works?filter=awards.id:G', abs(xxhash64(CONCAT(f.funder_id, ':', LOWER(g.grant_id)))) % 9000000000) as works_api_url,

        -- Timestamps
        current_timestamp() as created_date,
        current_timestamp() as updated_date

    FROM openalex.awards.gates_raw g
    CROSS JOIN gates_funder f
    WHERE g.grant_id IS NOT NULL
      AND TRIM(g.grant_id) != ''
)

SELECT * FROM awards_transformed;

In [ ]:
%sql
-- Remove previous data for this source before inserting fresh data
DELETE FROM openalex.awards.openalex_awards_raw
WHERE provenance = 'gates_foundation' AND priority = 4;

-- Insert into openalex_awards_raw with priority
INSERT INTO openalex.awards.openalex_awards_raw
SELECT
    id,
    display_name,
    description,
    funder_id,
    funder_award_id,
    amount,
    currency,
    funder,
    funding_type,
    funder_scheme,
    provenance,
    start_date,
    end_date,
    start_year,
    end_year,
    lead_investigator,
    co_lead_investigator,
    investigators,
    landing_page_url,
    doi,
    works_api_url,
    created_date,
    updated_date,
    4 as priority
FROM openalex.awards.gates_awards;

## Verification Queries

In [ ]:
%sql
-- Check row count (should be ~40K)
SELECT COUNT(*) as total_gates_awards FROM openalex.awards.gates_awards;

In [ ]:
%sql
-- Sample the data
SELECT 
    id,
    display_name,
    funder_award_id,
    funder_scheme,
    funding_type,
    amount,
    currency,
    start_date,
    end_date,
    lead_investigator.affiliation.name as grantee,
    lead_investigator.affiliation.country as country
FROM openalex.awards.gates_awards 
LIMIT 10;

In [ ]:
%sql
-- Check funder distribution (should all be Gates Foundation)
SELECT funder.display_name, COUNT(*) as cnt
FROM openalex.awards.gates_awards
GROUP BY funder.display_name
ORDER BY cnt DESC;

In [ ]:
%sql
-- Check funding_type distribution
SELECT funding_type, COUNT(*) as cnt
FROM openalex.awards.gates_awards
GROUP BY funding_type
ORDER BY cnt DESC;

In [ ]:
%sql
-- Check funder_scheme distribution (divisions)
SELECT funder_scheme, COUNT(*) as cnt, 
       ROUND(SUM(amount)/1e9, 2) as total_amount_billions
FROM openalex.awards.gates_awards
WHERE funder_scheme IS NOT NULL
GROUP BY funder_scheme
ORDER BY cnt DESC
LIMIT 20;

In [ ]:
%sql
-- Check data completeness
SELECT
    COUNT(*) as total,
    COUNT(display_name) as has_title,
    COUNT(description) as has_description,
    COUNT(amount) as has_amount,
    COUNT(start_date) as has_start_date,
    COUNT(end_date) as has_end_date,
    COUNT(lead_investigator) as has_grantee_info,
    ROUND(COUNT(display_name) * 100.0 / COUNT(*), 1) as pct_title,
    ROUND(COUNT(amount) * 100.0 / COUNT(*), 1) as pct_amount,
    ROUND(SUM(amount)/1e9, 2) as total_amount_billions
FROM openalex.awards.gates_awards;

In [ ]:
%sql
-- Check year distribution
SELECT start_year, COUNT(*) as cnt,
       ROUND(SUM(amount)/1e9, 2) as total_amount_billions
FROM openalex.awards.gates_awards
WHERE start_year IS NOT NULL
GROUP BY start_year
ORDER BY start_year DESC
LIMIT 20;

In [ ]:
%sql
-- Top grantee organizations
SELECT 
    lead_investigator.affiliation.name as grantee,
    lead_investigator.affiliation.country as country,
    COUNT(*) as grant_count,
    ROUND(SUM(amount)/1e6, 1) as total_amount_millions
FROM openalex.awards.gates_awards
WHERE lead_investigator.affiliation.name IS NOT NULL
GROUP BY lead_investigator.affiliation.name, lead_investigator.affiliation.country
ORDER BY total_amount_millions DESC
LIMIT 25;

In [ ]:
%sql
-- Countries by total funding
SELECT 
    lead_investigator.affiliation.country as country,
    COUNT(*) as grant_count,
    ROUND(SUM(amount)/1e9, 2) as total_amount_billions
FROM openalex.awards.gates_awards
WHERE lead_investigator.affiliation.country IS NOT NULL
GROUP BY lead_investigator.affiliation.country
ORDER BY total_amount_billions DESC
LIMIT 20;